In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import cm
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from numpy import linalg as la
from numpy import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from solver_sdmd_torch_gpu2 import KoopmanNNTorch, KoopmanSolverTorch

from numpy import random as rrr
from time import time
from koopman_bandit import KoopmanBandit
import joblib
from collections import namedtuple, deque
import math
import random as rnd
from scipy import stats

In [ ]:
print (torch.__version__, torch.cuda.is_available())
print(torch.version.cuda)
print (torch.cuda.get_device_name())
#device= 'cpu'
device= 'cuda'

In [ ]:
## Choose a system (this is just for name)
system = 'duffing' 
# system = 'double_well'

In [ ]:
# reward_hist= joblib.load (f'dqn_history32x32_12000steps_example_{system}_final.jbl')
# os.makedirs(f'figures_dqn_{system}', exist_ok=True)

In [ ]:
# #define the domain of the dynamical system
# x_min= -3
# x_max= 3
# y_min = -4
# y_max= 4

# from double_well_potential_2d import potential, simulate_trajectory

# """
# Main script for plotting and trajectory generation.
# Imports potential and simulate_trajectory from double_well.py.
# """

# # Plot the potential landscape
# x_plot = np.linspace(-2, 2, 100)
# y_plot = np.linspace(-3, 3, 100)
# XX, YY = np.meshgrid(x_plot, y_plot)
# ZZ = potential(XX, YY)

# plt.figure(figsize=(6, 4))
# plt.contourf(XX, YY, ZZ, levels=20, cmap='coolwarm')
# plt.colorbar(label='Potential')
# plt.title('2D Double Well Potential Landscape')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.show()

# # Generate a single SDE trajectory

# def get_single_trajectory(x0, y0, T):
#     """
#     Produce SDE trajectory starting at (x0, y0) over T steps.
#     This function delegates the simulation to double_well.simulate_trajectory.
#     """
#     return simulate_trajectory(x0, y0, T)

In [ ]:
from duffing_system_2d import simulate_trajectory

"""
Main script for Duffing oscillator system simulation
"""

# Set parameters for the Duffing oscillator
delta = 0.5
alpha = -1
beta = 1
gamma = 0
omega = 0
sigma = 0.15

# Create a grid of points for phase space visualization
x_plot = np.linspace(-2, 2, 100)
y_plot = np.linspace(-2, 2, 100)
XX, YY = np.meshgrid(x_plot, y_plot)

# Calculate vector field for phase space plot
U = YY  # dx/dt = y
V = -delta*YY - alpha*XX - beta*(XX**3) + gamma*np.cos(omega*0)  # dy/dt at t=0

# Normalize the arrows
magnitude = np.sqrt(U**2 + V**2)
U_norm = U / magnitude
V_norm = V / magnitude

# Plot the phase space
plt.figure(figsize=(6, 4))
plt.streamplot(XX, YY, U, V, density=1.5, color='darkblue', linewidth=0.7)
plt.quiver(XX[::10, ::10], YY[::10, ::10], U_norm[::10, ::10], V_norm[::10, ::10], 
           color='red', scale=25)
plt.title('Duffing Oscillator Phase Space')
plt.xlabel('x (position)')
plt.ylabel('y (velocity)')
plt.grid(alpha=0.3)
plt.show()

def get_single_trajectory(x0, y0, T):
    """
    Produce SDE trajectory starting at (x0, y0) over T steps.
    This function delegates the simulation to duffing_system_2d.simulate_trajectory.
    
    Args:
        x0: Initial x-coordinate (position)
        y0: Initial y-coordinate (velocity)
        T: Total simulation time steps
        
    Returns:
        Tuple of (data_matrix_single, lag_time)
    """
    return simulate_trajectory(x0, y0, T, delta=delta, alpha=alpha, beta=beta, gamma=gamma, omega=omega)

In [ ]:
def koopman_reward_func (action, k_grid= 4):
    action_x= action//k_grid
    action_y= action%k_grid
    x_borders= linspace (-3,3, k_grid+1)
    y_borders= linspace (-4,4, k_grid+1)
    x_lo= x_borders[action_x]
    x_hi= x_borders[action_x+1]
    y_lo= y_borders[action_y]
    y_hi= y_borders[action_y+1]
    x0_single= rrr.uniform (x_lo, x_hi)
    y0_single= rrr.uniform (y_lo, y_hi)
    
    T= 600 # 1000
    
    data_matrix_single, lag_time= get_single_trajectory (x0_single, y0_single, T)
    # Extract data_X and data_Y from the data matrix
    data_X = data_matrix_single[:, :-1, :]
    data_Y = data_matrix_single[:, 1:, :]
    print(f"Shape of data_X: {data_X.shape}")
    print(f"Shape of data_Y: {data_Y.shape}")
    
    # Reshape data_X and data_Y into a single column
    X = data_X.reshape(-1, data_X.shape[2])  # 2D features
    Y = data_Y.reshape(-1, data_X.shape[2])  # 2D targets
    print(f"Shape of X: {X.shape}")
    print(f"Shape of Y: {Y.shape}")
    
    # Separate data into two parts: train and validation
    len_all = X.shape[0]
    data_x_train = X[:int(0.7*len_all)]
    data_x_valid = X[int(0.7*len_all)+1:]
    
    data_y_train = Y[:int(0.7*len_all)]
    data_y_valid = Y[int(0.7*len_all)+1:]
    
    data_train = [data_x_train, data_y_train]
    data_valid = [data_x_valid, data_y_valid]
    
    print(data_x_train.shape)
    
    checkpoint_file= f'example_{system}_dqn_ckpt.torch'
    basis_function = KoopmanNNTorch(input_size= 2, layer_sizes=[10], n_psi_train=10).to(device)  # basis number would be 20
    
    
    solver = KoopmanSolverTorch(dic=basis_function, # Replace 'koopman_nn' by 'dic' if you use the original solver_edmdvar
                           target_dim=np.shape(data_x_train)[-1],
                                                       reg=0.1,  checkpoint_file= checkpoint_file, fnn_checkpoint_file= f'example_{system}_fnn_dqn.torch', 
                                a_b_file= f'sde_coefficients_example_{system}_dqn.jbl', 
                            generator_batch_size= 2, fnn_batch_size= 32, delta_t= lag_time)
    solver.build_with_generator(
    data_train=data_train,
    data_valid=data_valid,
    epochs=4,
    batch_size=256,
    lr=1e-5,
    log_interval=10,
    lr_decay_factor=.8
    )
    consistency= 0
    for ii in arange (len (solver.eigenvalues)):
        phi_x= solver.eigenfunctions (X)[ii]
        phi_y=  solver.eigenfunctions (Y)[ii]
        lmbd=  solver.eigenvalues[ii]
        consistency= consistency + la.norm (phi_y- lmbd*phi_x)**2
    print (consistency)
    
    return 16-consistency

def get_ee_reward (trajectory,next_initial_point):
    ee_epsilon= 0.1
    kernel = stats.gaussian_kde(trajectory.squeeze().T)
    mean_kernel= np.mean (kernel(trajectory.squeeze().T))
    normalized_p= kernel (next_initial_point)/mean_kernel
    ee_reward= 1.0/(normalized_p + ee_epsilon)
    return ee_reward

def do_koopman_step (state, action, k_grid= 4, state_len= 10, chunk_len= 100):
    chunk_list= []
    T= chunk_len
    action_x= action//k_grid
    action_y= action%k_grid
    for ii in arange(state_len-1):
        
        x0_single= state[0, ii]
        y0_single=  state[1, ii]
   
        data_matrix_single, lag_time= get_single_trajectory (x0_single, y0_single, T)
        chunk_list.append (data_matrix_single)
    x_borders= linspace (-3,3, k_grid+1)
    y_borders= linspace (-4,4, k_grid+1)
    x_lo= x_borders[action_x]
    x_hi= x_borders[action_x+1]
    y_lo= y_borders[action_y]
    y_hi= y_borders[action_y+1]
    x0_single= rrr.uniform (x_lo, x_hi)
    y0_single= rrr.uniform (y_lo, y_hi)
    data_matrix_single, lag_time= get_single_trajectory (x0_single, y0_single, T)
    trajectory= hstack (chunk_list)
    chunk_list.append (data_matrix_single)
    data_matrix_single= hstack (chunk_list)
    ee_reward= get_ee_reward (trajectory, [x0_single, y0_single])
    #print ('EE reward:' , ee_reward)
    #Extract data_X and data_Y from the data matrix
    data_X = data_matrix_single[:, :-1, :]
    data_Y = data_matrix_single[:, 1:, :]
    print(f"Shape of data_X: {data_X.shape}")
    print(f"Shape of data_Y: {data_Y.shape}")
    
    # Reshape data_X and data_Y into a single column
    X = data_X.reshape(-1, data_X.shape[2])  # 2D features
    Y = data_Y.reshape(-1, data_X.shape[2])  # 2D targets
    print(f"Shape of X: {X.shape}")
    print(f"Shape of Y: {Y.shape}")
    
    # Separate data into two parts: train and validation
    len_all = X.shape[0]
    data_x_train = X[:int(0.7*len_all)]
    data_x_valid = X[int(0.7*len_all)+1:]
    
    data_y_train = Y[:int(0.7*len_all)]
    data_y_valid = Y[int(0.7*len_all)+1:]
    
    data_train = [data_x_train, data_y_train]
    data_valid = [data_x_valid, data_y_valid]
    
    print(data_x_train.shape)
    
    checkpoint_file= f'example_{system}_dqn_ckpt3.torch'
    basis_function = KoopmanNNTorch(input_size= 2, layer_sizes=[10], n_psi_train=10).to(device)  # basis number would be 20
    
    
    solver = KoopmanSolverTorch(dic=basis_function, # Replace 'koopman_nn' by 'dic' if you use the original solver_edmdvar
                           target_dim=np.shape(data_x_train)[-1],
                                                       reg=0.1,  checkpoint_file= checkpoint_file, fnn_checkpoint_file= f'example_{system}_fnn_dqn1.torch', 
                                a_b_file= f'sde_coefficients_example_{system}_dqn1.jbl', 
                            generator_batch_size= 2, fnn_batch_size= 32, delta_t= lag_time)
    solver.build_with_generator(
    data_train=data_train,
    data_valid=data_valid,
    epochs=4,
    batch_size=256,
    lr=1e-5,
    log_interval=10,
    lr_decay_factor=.8
    )
    consistency= 0
    for ii in arange (len (solver.eigenvalues)):
        phi_x= solver.eigenfunctions (X)[ii]
        phi_y=  solver.eigenfunctions (Y)[ii]
        lmbd=  solver.eigenvalues[ii]
        consistency= consistency + la.norm (phi_y- lmbd*phi_x)**2
    print (consistency)
    reward= 16- consistency+  0.15*ee_reward
    next_state= hstack ([state[:, 1:], expand_dims (array ([x0_single, y0_single]), axis= 1)])
    return next_state, reward
        
    



Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return rnd.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
batch_size= 64
gamma = 0.99
eps_start = 0.9
eps_end = 0.2 # 0.2
eps_decay = 2000
tau = 0.05
lr = 1e-4

k_grid= 32
state_len= 12
chunk_len= 100


In [ ]:
n_actions = k_grid**2
# Get the number of state observations

n_observations = state_len*2

In [ ]:
policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=lr, amsgrad=True)
memory = ReplayMemory(20000)


steps_done = 0

In [ ]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = eps_end + (eps_start - eps_end) * \
        math.exp(-1. * steps_done / eps_decay)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            state_tensor= torch.tensor(state.ravel(), dtype=torch.float64, device=device).unsqueeze(0)
            #state_tensor= torch.tensor (state.ravel(), dtype=torch.float64).to (device)
            return policy_net(state_tensor).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[np.random.choice(arange(n_actions))]], device=device, dtype=torch.long)

In [ ]:
def optimize_model():
    if len(memory) < batch_size:
        return
    transitions = memory.sample(batch_size)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(batch_size, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * gamma) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
#Below, you can find the main training loop. At the beginning we reset the environment and obtain the initial state Tensor. Then, we sample an action, execute it, observe the next state and the reward (always 1), and optimize our model once. When the episode ends (our model fails), we restart the loop.

#Below, num_episodes is set to 600 if a GPU is available, otherwise 50 episodes are scheduled so training does not take too long. However, 50 episodes is insufficient for to observe good performance on CartPole. You should see the model constantly achieve 500 steps within 600 training episodes. Training RL agents can be a noisy process, so restarting training can produce better results if convergence is not observed.

In [ ]:
# num_episodes= 1
# reward_hist= []
# num_steps= 12000 # 15000
# for i_episode in range(num_episodes):
#     # Initialize the environment and get its state
#     initial_state= 2*(np.random.uniform(size=(2,state_len))-1)
#     state= initial_state
#     for t in arange(num_steps):
#         initial_state= 2*(np.random.uniform(size=(2,state_len))-1)
#         action_tensor = select_action(state)
#         action= action_tensor.detach().cpu().numpy()
#         observation, reward= do_koopman_step (state, int (action.squeeze()), k_grid= k_grid)
#         reward_tensor = torch.tensor([reward], device=device)
#         #done = terminated or truncated

#         state_tensor= torch.tensor(state.ravel(), dtype=torch.float64, device=device).unsqueeze(0)
#         next_state = observation #torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
#         next_state_tensor= torch.tensor(observation.ravel(), dtype=torch.float64, device=device).unsqueeze(0)
#         # Store the transition in memory
#         memory.push(state_tensor, action_tensor, next_state_tensor, reward_tensor)
#         reward_hist.append ([state, action, next_state, reward])
#         # Move to the next state
#         state = next_state

#         # Perform one step of the optimization (on the policy network)
#         optimize_model()

#         # Soft update of the target network's weights
#         # θ′ ← τ θ + (1 −τ )θ′
#         target_net_state_dict = target_net.state_dict()
#         policy_net_state_dict = policy_net.state_dict()
#         for key in policy_net_state_dict:
#             target_net_state_dict[key] = policy_net_state_dict[key]*tau + target_net_state_dict[key]*(1-tau)
#         target_net.load_state_dict(target_net_state_dict)
        
#         # np.savetxt ('dqn_state.txt', [int(t)])
#         with open('dqn_state.txt', 'w') as f:
#             f.write(f"current RL step: {int(t)}\n")
            
#         if (t%100==0):
            
#             # joblib.dump (reward_hist, 'dqn_history64x64_example.jbl')
#             if (t % 100 == 0):
#                 joblib.dump(
#                     reward_hist,
#                     f'dqn_history{k_grid}x{k_grid}_{num_steps}steps_example_double_well.jbl'
#                 )


# print('Complete')

In [ ]:
import time

# Start timing the execution
start_time = time.time()

num_episodes = 1
reward_hist = []
num_steps = 15000  # 12000

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    initial_state = 2 * (np.random.uniform(size=(2, state_len)) - 1)
    state = initial_state
    
    for t in range(num_steps):
        # Record current time for time estimation
        current_time = time.time()
        elapsed_time = current_time - start_time
        
        # Calculate estimated time remaining
        if t > 0:
            avg_time_per_step = elapsed_time / t
            remaining_steps = num_steps - t
            estimated_time_remaining = avg_time_per_step * remaining_steps
        else:
            estimated_time_remaining = 0
        
        initial_state = 2 * (np.random.uniform(size=(2, state_len)) - 1)
        action_tensor = select_action(state)
        action = action_tensor.detach().cpu().numpy()
        observation, reward = do_koopman_step(state, int(action.squeeze()), k_grid=k_grid)
        reward_tensor = torch.tensor([reward], device=device)
        
        state_tensor = torch.tensor(state.ravel(), dtype=torch.float64, device=device).unsqueeze(0)
        next_state = observation
        next_state_tensor = torch.tensor(observation.ravel(), dtype=torch.float64, device=device).unsqueeze(0)
        
        # Store the transition in memory
        memory.push(state_tensor, action_tensor, next_state_tensor, reward_tensor)
        reward_hist.append([state, action, next_state, reward])
        
        # Move to the next state
        state = next_state
        
        # Perform one step of the optimization (on the policy network)
        optimize_model()
        
        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key] * tau + target_net_state_dict[key] * (1 - tau)
        target_net.load_state_dict(target_net_state_dict)
        
        # Format time strings for better readability
        def format_time(seconds):
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            seconds = int(seconds % 60)
            return f"{hours:02d}h:{minutes:02d}m:{seconds:02d}s"
        
        # Write detailed status to file
        with open(f'dqn_state_{system}.txt', 'w') as f:
            f.write(f"System: {system}\n")
            f.write(f"Current RL step: {int(t)}/{num_steps} ({(t/num_steps)*100:.1f}%)\n")
            f.write(f"Elapsed time: {format_time(elapsed_time)}\n")
            f.write(f"Estimated time remaining: {format_time(estimated_time_remaining)}\n")
        
        # Periodically save results
        if (t % 100 == 0):
            joblib.dump(
                reward_hist,
                f'dqn_history{k_grid}x{k_grid}_{num_steps}steps_example_{system}.jbl'
            )
            print(f"Step {t}/{num_steps} completed. System: {system}. Elapsed: {format_time(elapsed_time)}")

# Record total execution time
total_time = time.time() - start_time
print(f'Complete! Total execution time: {format_time(total_time)}')

In [ ]:
rewards_all=[]
states_all= []
actions_all= []
next_states_all= []
for ii in arange(len (reward_hist)):
        curr_list= reward_hist[ii]
        rewards_all.append (curr_list[-1])
        states_all.append (curr_list[0])
        actions_all.append (int (curr_list[1].squeeze()))
        next_states_all.append (curr_list[2])
        

In [ ]:
# joblib.dump (reward_hist, 'dqn_history64x64_15000steps_example_final.jbl')
filename = f'dqn_history{k_grid}x{k_grid}_{num_steps}steps_example_{system}_final.jbl'
joblib.dump(
    reward_hist,
    filename
)